# Laboratorio 8
Datos de ["Store Item Demand Forecasting Challenge"](https://www.kaggle.com/competitions/demand-forecasting-kernels-only/data)

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import os

### Carga y preparación de datos

In [2]:
# usamos solo los datos en train para validar las respuestas luego
df = pd.read_csv("data/train.csv", parse_dates=["date"])

# Revisión de estructura
print(f"Datos cargados: {df.shape}")
print("\nPrimeras filas:")
print(df.head())

# Convertir fecha a datetime
df['date'] = pd.to_datetime(df['date'])

print("\nInformación del dataset:")
print(df.info())

# Revisar valores faltantes
print("\nValores faltantes")
print(df.isna().sum())

Datos cargados: (913000, 4)

Primeras filas:
        date  store  item  sales
0 2013-01-01      1     1     13
1 2013-01-02      1     1     11
2 2013-01-03      1     1     14
3 2013-01-04      1     1     13
4 2013-01-05      1     1     10

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    913000 non-null  datetime64[ns]
 1   store   913000 non-null  int64         
 2   item    913000 non-null  int64         
 3   sales   913000 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 27.9 MB
None

Valores faltantes
date     0
store    0
item     0
sales    0
dtype: int64


In [3]:
# ya que no hay datos faltantes en el dataset no es necesario trabajarlo

# Verificar duplicados
duplicados = df.duplicated().sum()
print(f"Filas duplicadas: {duplicados}")
if duplicados > 0:
    df = df.drop_duplicates()
    
# Estadísticas básicas
print(f"\nEstadísticas de ventas:")
print(df['sales'].describe())
    
# Detectar outliers usando IQR
Q1 = df['sales'].quantile(0.15)
Q3 = df['sales'].quantile(0.85)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = df[(df['sales'] < lower_bound) | (df['sales'] > upper_bound)]
print(f"\nOutliers detectados: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")

# Capping en lugar de eliminación
df['sales'] = df['sales'].clip(lower=lower_bound, upper=upper_bound)

Filas duplicadas: 0

Estadísticas de ventas:
count    913000.000000
mean         52.250287
std          28.801144
min           0.000000
25%          30.000000
50%          47.000000
75%          70.000000
max         231.000000
Name: sales, dtype: float64

Outliers detectados: 357 (0.04%)


In [4]:
# Ver resultado de quitar outliers
print(df.describe())

# Ordenar por fecha
df = df.sort_values('date').reset_index(drop=True)

                                date          store           item  \
count                         913000  913000.000000  913000.000000   
mean   2015-07-02 11:59:59.999999744       5.500000      25.500000   
min              2013-01-01 00:00:00       1.000000       1.000000   
25%              2014-04-02 00:00:00       3.000000      13.000000   
50%              2015-07-02 12:00:00       5.500000      25.500000   
75%              2016-10-01 00:00:00       8.000000      38.000000   
max              2017-12-31 00:00:00      10.000000      50.000000   
std                              NaN       2.872283      14.430878   

               sales  
count  913000.000000  
mean       52.246664  
min         0.000000  
25%        30.000000  
50%        47.000000  
75%        70.000000  
max       173.000000  
std        28.784893  


In [5]:
# Transformación de datos
# Codificar store e item
store_enc = LabelEncoder()
item_enc = LabelEncoder()
df["store_id"] = store_enc.fit_transform(df["store"])
df["item_id"] = item_enc.fit_transform(df["item"])

n_stores = df["store_id"].nunique()
n_items = df["item_id"].nunique()

In [6]:
# Normalizamos ventas por serie
scalers = {}  # un scaler por serie
for s in range(n_stores):
    for i in range(n_items):
        mask = (df["store_id"]==s) & (df["item_id"]==i)
        scaler = MinMaxScaler()
        df.loc[mask, "sales_scaled"] = scaler.fit_transform(df.loc[mask, ["sales"]])
        scalers[(s,i)] = scaler

### Preprocesamiento de datos

In [7]:
# Obtener fechas únicas ordenadas
fechas_unicas = sorted(df['date'].unique())
print(f"Rango de fechas: {fechas_unicas[0]} a {fechas_unicas[-1]}")
print(f"Total de días: {len(fechas_unicas)}")

Rango de fechas: 2013-01-01 00:00:00 a 2017-12-31 00:00:00
Total de días: 1826


In [8]:
# Definir puntos de corte
test_days = 90  # 3 meses
val_days = 90   # 3 meses

fecha_test_inicio = fechas_unicas[-test_days]
fecha_val_inicio = fechas_unicas[-(test_days + val_days)]

print(f"\nDivisión temporal:")
print(f"Train: {fechas_unicas[0]} a {fechas_unicas[-(test_days + val_days + 1)]}")
print(f"Validación: {fecha_val_inicio} a {fechas_unicas[-(test_days + 1)]}")
print(f"Test: {fecha_test_inicio} a {fechas_unicas[-1]}")

# Crear conjuntos
df_train = df[df['date'] < fecha_val_inicio].copy()
df_val = df[(df['date'] >= fecha_val_inicio) & (df['date'] < fecha_test_inicio)].copy()
df_test = df[df['date'] >= fecha_test_inicio].copy()

print(f"\nTamaños de conjuntos:")
print(f"Train: {len(df_train)} registros")
print(f"Validación: {len(df_val)} registros")
print(f"Test: {len(df_test)} registros")


División temporal:
Train: 2013-01-01 00:00:00 a 2017-07-04 00:00:00
Validación: 2017-07-05 00:00:00 a 2017-10-02 00:00:00
Test: 2017-10-03 00:00:00 a 2017-12-31 00:00:00

Tamaños de conjuntos:
Train: 823000 registros
Validación: 45000 registros
Test: 45000 registros


In [9]:
# Generación de secuencias
# Ventana histórica de 90 días para predecir los próximos 90 días (3 meses)

lookback = 90  # días históricos para usar
forecast = 90  # días a predecir (3 meses)

def create_sequences(data, store_id, item_id, lookback, forecast):
    """
    Crea secuencias de entrada (X) y salida (y) para una serie temporal.
    
    Args:
        data: DataFrame completo
        store_id: ID de la tienda
        item_id: ID del item
        lookback: días históricos a usar
        forecast: días a predecir
    
    Returns:
        X: array con secuencias de entrada (n_samples, lookback)
        y: array con secuencias de salida (n_samples, forecast)
        dates: fechas correspondientes a cada secuencia
    """
    # Filtrar datos para esta combinación store-item
    mask = (data['store_id'] == store_id) & (data['item_id'] == item_id)
    serie = data[mask].sort_values('date')
    
    # Extraer ventas escaladas
    values = serie['sales_scaled'].values
    dates = serie['date'].values
    
    X, y, seq_dates = [], [], []
    
    # Crear ventanas deslizantes
    for i in range(len(values) - lookback - forecast + 1):
        X.append(values[i:i + lookback])
        y.append(values[i + lookback:i + lookback + forecast])
        seq_dates.append(dates[i + lookback - 1])  # fecha del último dato de entrada
    
    return np.array(X), np.array(y), np.array(seq_dates)


# Generar secuencias para todos los store-item
X_train_list, y_train_list = [], []
X_val_list, y_val_list = [], []
X_test_list, y_test_list = [], []

print("\nGenerando secuencias...")
for store_id in range(n_stores):
    for item_id in range(n_items):
        # Secuencias de entrenamiento
        X_tr, y_tr, _ = create_sequences(df_train, store_id, item_id, lookback, forecast)
        if len(X_tr) > 0:
            X_train_list.append(X_tr)
            y_train_list.append(y_tr)
        
        # Secuencias de validación (usar train + val para crear ventanas)
        df_train_val = pd.concat([df_train, df_val])
        X_v, y_v, dates_v = create_sequences(df_train_val, store_id, item_id, lookback, forecast)
        # Filtrar solo secuencias que predicen en periodo de validación
        mask_val = dates_v >= fecha_val_inicio - pd.Timedelta(days=lookback)
        if mask_val.sum() > 0:
            X_val_list.append(X_v[mask_val])
            y_val_list.append(y_v[mask_val])
        
        # Secuencias de test (usar train + val + test para crear ventanas)
        df_full = pd.concat([df_train, df_val, df_test])
        X_t, y_t, dates_t = create_sequences(df_full, store_id, item_id, lookback, forecast)
        # Filtrar solo secuencias que predicen en periodo de test
        mask_test = dates_t >= fecha_test_inicio - pd.Timedelta(days=lookback)
        if mask_test.sum() > 0:
            X_test_list.append(X_t[mask_test])
            y_test_list.append(y_t[mask_test])

# Concatenar todas las secuencias
X_train = np.concatenate(X_train_list, axis=0)
y_train = np.concatenate(y_train_list, axis=0)

X_val = np.concatenate(X_val_list, axis=0)
y_val = np.concatenate(y_val_list, axis=0)

X_test = np.concatenate(X_test_list, axis=0)
y_test = np.concatenate(y_test_list, axis=0)

print(f"\nForma de los conjuntos generados:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

print("\nPreparación de datos completada ✓")


Generando secuencias...

Forma de los conjuntos generados:
X_train: (733500, 90), y_train: (733500, 90)
X_val: (45000, 90), y_val: (45000, 90)
X_test: (45000, 90), y_test: (45000, 90)

Preparación de datos completada ✓


#### Descripción de las secuencias generadas.
Hemos creado un conjunto de datos con arquitectura secuencia a secuencia (seq2seq), donde cada muestra contiene:
* Entrada (X): 90 días consecutivos de ventas históricas (normalizadas)
* Salida (y): Los siguientes 90 días de ventas que queremos predecir

Usamos una ventana deslizante que se mueve día a día. Esto significa que de una serie temporal larga se generan múltiples ejemplos de entrenamiento:
* Secuencia 1: días 1-90 → predice días 91-180
* Secuencia 2: días 2-91 → predice días 92-181
* Secuencia 3: días 3-92 → predice días 93-182
* Y así sucesivamente

Al usar esta estructura, No se predice un solo valor sino 90 días completos, lo cual es más útil para planificación empresarial.
- 90 días de historia capturan múltiples ciclos semanales y mensuales
- Al normalizar por cada combinación store-item, el modelo aprende patrones relativos, no valores absolutos

#### ¿Para qué modelo es útil esta estructura?
1. LSTM (Long Short-Term Memory)
- Diseñado específicamente para secuencias temporales, permite capturar patrones de largo plazo como estacionalidad, puede "recordar" tendencias de semanas o meses atrás. Ideal para 90 días de entrada → 90 días de salida
2. Transformer con Attention
- Puede identificar qué días pasados son más relevantes para predecir días futuros. Captura relaciones complejas (ej: "las ventas del lunes dependen del lunes anterior")
3. Seq2Seq con Encoder-Decoder
- Es la arquitectura clásica para este tipo de problemas, pues es muy intuitivo conceptualmente
- Encoder: comprime los 90 días de entrada en una representación
- Decoder: genera los 90 días de predicción paso a paso

### Modelo LSTM

In [10]:
X_train_nn = X_train[..., np.newaxis]
X_val_nn = X_val[..., np.newaxis]
X_test_nn = X_test[..., np.newaxis]

y_train_nn = y_train[..., np.newaxis]
y_val_nn = y_val[..., np.newaxis]
y_test_nn = y_test[..., np.newaxis]

print("Shapes (after reshape):", X_train_nn.shape, y_train_nn.shape)

Shapes (after reshape): (733500, 90, 1) (733500, 90, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

timesteps = lookback
n_features = 1
output_steps = forecast

tf.keras.backend.clear_session()
latent_dim = 128  

# Encoder
encoder_inputs = Input(shape=(timesteps, n_features), name="encoder_input")
x = BatchNormalization()(encoder_inputs)
x = LSTM(latent_dim, return_sequences=False, name="encoder_lstm")(x)
x = Dropout(0.2)(x)
encoder_output = Dense(latent_dim, activation='relu', name="encoder_dense")(x)

# Repeat
decoder_input = RepeatVector(output_steps, name="repeat_vector")(encoder_output)

# Decoder 
decoder_lstm = LSTM(latent_dim, return_sequences=True, name="decoder_lstm")
x = decoder_lstm(decoder_input)
x = Dropout(0.2)(x)
x = TimeDistributed(Dense(64, activation='relu'))(x)
decoder_output = TimeDistributed(Dense(1), name="decoder_output")(x)

model = Model(encoder_inputs, decoder_output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 90, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 90, 1)          │             4 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_lstm (LSTM)             │ (None, 128)            │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_dense (Dense)           │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 90, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_lstm (LSTM)             │ (None, 90, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 90, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 90, 64)         │         8,256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_output                  │ (None, 90, 1)          │            65 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 222,981 (871.02 KB)

 Trainable params: 222,979 (871.01 KB)

 Non-trainable params: 2 (8.00 B)

In [16]:
from tensorflow.keras.layers import GRU

latent_dim = 128  

# Encoder
encoder_inputs = Input(shape=(timesteps, n_features), name="encoder_input")
x = BatchNormalization()(encoder_inputs)
x = GRU(latent_dim, return_sequences=False, name="encoder_gru")(x)
x = Dropout(0.2)(x)
encoder_output = Dense(latent_dim, activation='relu', name="encoder_dense")(x)

# Repeat
decoder_input = RepeatVector(output_steps, name="repeat_vector")(encoder_output)

# Decoder
decoder_gru = GRU(latent_dim, return_sequences=True, name="decoder_gru")
x = decoder_gru(decoder_input)
x = Dropout(0.2)(x)
x = TimeDistributed(Dense(64, activation='relu'))(x)
decoder_output = TimeDistributed(Dense(1), name="decoder_output")(x)

model = Model(encoder_inputs, decoder_output)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 90, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 90, 1)          │             4 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_gru (GRU)               │ (None, 128)            │        50,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_dense (Dense)           │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 90, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_gru (GRU)               │ (None, 90, 128)        │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 90, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 90, 64)         │         8,256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_output                  │ (None, 90, 1)          │            65 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 174,213 (680.52 KB)

 Trainable params: 174,211 (680.51 KB)

 Non-trainable params: 2 (8.00 B)

In [ ]:
output_dir = "models_lab8"
os.makedirs(output_dir, exist_ok=True)
chk_path = os.path.join(output_dir, "best_seq2seq_lstm.h5")

callbacks = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6),
    ModelCheckpoint(chk_path, monitor='val_loss', save_best_only=True, verbose=1)
]

batch_size = 256
epochs = 50

history = model.fit(
    X_train_nn, y_train_nn,
    validation_data=(X_val_nn, y_val_nn),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/2


KeyboardInterrupt: 

In [17]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=6,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

checkpoint = ModelCheckpoint(
    'best_gru_model.h5',
    monitor='val_loss',
    save_best_only=True
)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=128,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=1
)

Epoch 1/50
  94/5731 ━━━━━━━━━━━━━━━━━━━━ 37:54 404ms/step - loss: 0.0443 - mae: 0.1616

KeyboardInterrupt: 